In [20]:
import pandas as pd

import os

DATA_DIR = "data"
train_data = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
X = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))

In [21]:
from surprise import Dataset, Reader

reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(train_data, reader)

In [22]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)

In [23]:
from surprise import SVDpp

svdpp = SVDpp(
    n_factors=150, n_epochs=250, lr_all=0.047, reg_all=0.01, cache_ratings=True
)
svdpp.fit(trainset)

In [6]:
from surprise import accuracy

accuracy.rmse(svdpp.test(testset))

RMSE: 0.7666


0.7665889798987596

In [7]:
from surprise import SVD

svd = SVDpp(n_factors=50, n_epochs=2500, lr_all=0.002, reg_all=0.2)
svd.fit(trainset)

In [8]:
accuracy.rmse(svd.test(testset))

RMSE: 0.7476


0.7476284136876696

In [19]:
import numpy as np

predictions = []
true_ratings = []

weights = [0.2, 0.8]
for user_id, item_id, rating in testset:
    prediction = (
        svdpp.predict(user_id, item_id).est * weights[0]
        + svd.predict(user_id, item_id).est * weights[1]
    )
    predictions.append(prediction)
    true_ratings.append(rating)

rmse = np.sqrt(((np.array(predictions) - np.array(true_ratings)) ** 2).mean())
rmse

0.7468569747500935

In [3]:
# svdPP + svd
import pandas as pd

svdpp = pd.read_csv("predictions/svdpp_1.csv")
svd = pd.read_csv("predictions/svd_1.csv")

# public score: 0.66472
new_values = (svdpp["rating"].values + svd["rating"].values) / 2
svdpp["rating"] = new_values
svdpp.to_csv("svd_1+svdpp_1.csv", index=False)